## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### 1. Изучение данных

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


#### В таблице не хватает данных по трудовому стажу в днях и ежемесячному доходу. Возможно, не все клиенты трудоустроены или имеют постояный доход. 

### 2. Обработка пропусков

In [3]:
df['days_employed'] = df.groupby('dob_years').transform(lambda x: x.fillna(x.median()))
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Предположим, что трудовой стаж 'days_employed' зависит от возраста 'dob_years'. Чтобы заполнить пропуски значений трудового стажа, сгруппируем таблицу по столбцу 'dob_years' и заполним пропуск медианным значением среди тех, значение столбца 'dob_years' которых совпадает с пропущенным. 
Значения пропусков в столбце 'total_income' заменим медианным, потому что напрямую на доход ни один из столбцов не влияет. 

### 3. Замена типа данных

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['debt'] = df['debt'].astype('bool')
df['total_income'] = df['total_income'].astype('int')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: bool(1), int64(6), object(5)
memory usage: 1.8+ MB


Стаж в днях и доход должны быть целочисленным значением. Перевели их в тип int64. Наличие задолженности в столбце 'debt' может принимать только 2 значения - 0 или 1, поэтому можно перевести этот столбец в булевый тип данных. Для изменения типа данных использовался метод astype(), так как в его аргументе можно указать требуемый тип данных. В данном случае метод to_numeric() не подходит, так как по умолчанию переводит данные в тип float. 

### 4. Обработка дубликатов

In [5]:
df['education'].str.lower()
df['family_status'].str.lower()
df['gender'].str.lower()
df['income_type'].str.lower()
df['purpose'].str.lower()

df.duplicated().sum()
df = df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null bool
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: bool(1), int64(6), object(5)
memory usage: 1.8+ MB


Перед тем как начать поиск дубликатов, перевели все столбцы со строковыми значениями в нижний регистр. Далее удалили дубликаты и переписали индексы, чтобы лишние значения не влияли на исследование. Для поиска дубликатов использовали метод duplicated(), для удаления - drop_duplicates(). Возможные причины появления дубликатов : человеческий фактор, техническая ошибка.

### 5. Лемматизация

In [6]:
#df['purpose'].unique()
from pymystem3 import Mystem
m = Mystem ()
lemmas =[]
for i in range (0, df.shape[0]):
    lemmas += m.lemmatize(df['purpose'][i])
from collections import Counter
Counter(lemmas)


Counter({'покупка': 5900,
         ' ': 33596,
         'жилье': 4461,
         '\n': 21471,
         'приобретение': 461,
         'автомобиль': 4308,
         'дополнительный': 907,
         'образование': 4014,
         'сыграть': 769,
         'свадьба': 2335,
         'операция': 2604,
         'с': 2918,
         'на': 2228,
         'проведение': 773,
         'для': 1290,
         'семья': 638,
         'недвижимость': 6353,
         'коммерческий': 1312,
         'жилой': 1231,
         'строительство': 1879,
         'собственный': 635,
         'подержать': 478,
         'свой': 2231,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1315,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 652,
         'ремонт': 607})

#### Самые распостраненные цели кредитов - недвижимость, автомобиль и образование.
#### В процессе лемматизации была использована библиотека PyMystem. После создания пустого списка с помощью цикла for были лемматизированы все значения столбца purpose таблицы df. Количество полученных лемм посчитаны с помощью Counter.

### 6. Категоризация данных

In [8]:
def purpose_category(purpose): 
    if ('жилье' in m.lemmatize(purpose)) or ('недвижимость' in m.lemmatize(purpose)) or('строительство' in m.lemmatize(purpose)) or ('жилой' in m.lemmatize(purpose)):
        return '1_жилье/недвижимость'
    if ('автомобиль' in m.lemmatize(purpose)) or ('подержанный' in m.lemmatize(purpose)):
        return '2_автомобиль'
    if ('образование' in m.lemmatize(purpose)):
        return '3_образование' 
    return '4_свадьба' 

df['purpose_category'] = df['purpose'].apply(purpose_category)
#print(df[['purpose','purpose_category']].head(20))    
df['purpose_category'].value_counts()

1_жилье/недвижимость    10814
2_автомобиль             4308
3_образование            4014
4_свадьба                2335
Name: purpose_category, dtype: int64

#### С помощью категоризации данных выяснили соотношение количества кредитов на различные цели. Для категоризации выбрали самые распространенные леммы и их аналоги.

### 7. Ответы на вопросы:

#### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [52]:
def children_num(children):
    if children == 0:
        return '0(бездетные)'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 ребенка'
    if children == 3:
        return '3 ребенка'
    return 'многодетные'
df['children_num']  = df['children'].apply(children_num) 

df['debt'] = df['debt'].astype('int')
table_children = df.pivot_table (index = 'children_num', values = 'debt')
print(table_children)

                  debt
children_num          
0(бездетные)  0.075353
1 ребенок     0.092327
2 ребенка     0.094542
3 ребенка     0.081818
многодетные   0.075145


* Больше всего задолженностей оказалось у людей, имеющих 1 или 2 детей. При этом количество задолженностей у бездетных и многодетных примерно равны. Таким образом прямой корреляции между наличием детей и возвратом кредита в срок не прослеживается. *

#### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [53]:
table_fam_status = df.pivot_table (index = 'family_status', values = 'debt')
print(table_fam_status)

                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093202
женат / замужем        0.075421


* Чаще всего задолженность встречается у людей с семейным положением "Не женат / не замужем" и людей состоящих в гражданском браке, а реже всего у вдов и вдовцов.

#### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
def income(total_income):
    if 0<= total_income < 75000:
        return '1_low'
    if 75000 <= total_income < 150000:
        return '2_mean'
    if 150000 <= total_income < 225000:
        return '3_high'
    if total_income >= 225000:
        return '4_very high'
    return 'incorrect'

df['total_income_new'] = df['total_income'].apply(income) 


table_income = df.pivot_table (index = 'purpose_category', values = 'debt')
print(table_income )

                      debt
purpose_category          
1                 0.072314
2                 0.093547
3                 0.092177
4                 0.079657


* Судя по всему, корреляции между уровнем дохода и возвратом долгов нет. Меньше всего задолженностей у людей сочень высоким доходом, больше всего у людей со средним. 

####  - Как разные цели кредита влияют на его возврат в срок?

In [61]:
table_purpose = df.pivot_table (index = 'purpose_category', values = 'debt')
print(table_purpose)

                          debt
purpose_category              
1_жилье/недвижимость  0.072314
2_автомобиль          0.093547
3_образование         0.092177
4_другое              0.079657


* Меньше всего залолженностей возникает, когда цель кредита связана с жильем/недвижимостью, большое количество долгов характерно для кредитов взятых на образование или автомобиль. 

### 8. Общий вывод

Таким образом, прямой зависимости между возвратом кредита с срок и наличием детей или уровнем дохода не прослеживается. Однако, такая связь присутствует с семейным положением (чаще всего задолженность встречается у людей с семейным положением "не женат / не замужем" и людей состоящих в гражданском браке, а реже всего у вдов и вдовцов)и целью кредита (меньше всего залолженностей возникает, когда цель кредита связана с жильем/недвижимостью, большое количество долгов характерно для кредитов взятых на образование или автомобиль).